In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
MATCH_CSV = "/goalimpacct/data_compressed/match/compressed.csv"
rawMatchDF =  pd.read_csv(MATCH_CSV)

In [ ]:
CACHED_DATA_DIR = "/goalimpacct/spark_data_cache/result_parquet"
rawData =  pd.read_parquet(CACHED_DATA_DIR)

In [ ]:
uniqueLeaguesDF = rawMatchDF.loc[:,[':tournament']].drop_duplicates()

In [ ]:
uniqueLeaguesDF

In [ ]:
guestDF = rawMatchDF.loc[:, [':tournament',':saison', ':guest']].rename(columns={':guest' : ":team"})

In [ ]:
homeDF = rawMatchDF.loc[:, [':tournament',':saison', ':home']].rename(columns={':home' : ":team"})

In [ ]:
teamDF = pd.concat([homeDF, guestDF]).drop_duplicates()
teamDF[':team'] = pd.to_numeric(teamDF[':team'], errors='coerce')
teamDF = teamDF.dropna(subset=[':team'])
teamDF[':team'] = teamDF[':team'].apply(np.int64)
teamDF[':team'] = teamDF[':team'].astype(str)
teamDF[':saison'] = teamDF[':saison'].astype(str)
teamDF

In [ ]:
teamSeasonScoreDF = rawData.groupby([':saison', ':team'])[':avgOffPointsLast1Matches'].agg(['mean'])
teamSeasonScoreDF = teamSeasonScoreDF.reset_index()

teamSeasonScoreDF

In [ ]:
tournamentScoresDF = teamSeasonScoreDF.merge(teamDF, on=[':saison', ':team'])
tournamentScoresDF = tournamentScoresDF.drop(columns=[':team'])
tournamentScoresDF

In [ ]:
topNTournamentScoresDF = tournamentScoresDF.groupby([':tournament', 
                            ':saison'])['mean'].nlargest(5).reset_index()[[':tournament', ':saison', 'mean']]

topNTournamentScoresDF.groupby([':tournament', 
                            ':saison'])['mean'].agg('mean')

